# Limpieza

Este cuaderno demuestra cómo limpiar todos los recursos creados en el conjunto anterior de cuadernos.

In [ ]:
import boto3
import json

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [ ]:
%store -r

Tenga en cuenta que estos pasos dependen de que cada uno de ellos se complete en su totalidad para que el siguiente funcione. Si recibe un mensaje de error, espere uno o dos minutos y vuelva a intentarlo.

## Limpiar campañas

Comience por limpiar las campañas. Es necesario eliminar las campañas antes de poder eliminar las versiones de la solución en las que se basan. 

El código que se muestra a continuación borrará las campañas creadas.

In [ ]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

## Limpiar rastreadores de eventos

Luego, limpie los rastreadores de eventos. El código que se muestra a continuación borrará el rastreador de eventos.

In [ ]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

## Limpiar filtros

Luego, limpie los filtros. El código que se muestra a continuación itera todos los filtros creados y los elimina. Tenga en cuenta que, si recibe un mensaje de error sobre una campaña, no pasa nada, espere 2 minutos y vuelva a intentarlo con la celda que aparece a continuación. 

Nota: No podrá eliminar los filtros hasta que se complete la eliminación de la campaña, lo que puede tardar unos minutos.

In [ ]:
for filter_arn in meta_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in decade_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## Limpiar soluciones

Luego, limpie las soluciones. El código que se muestra a continuación borrará las soluciones creadas.

In [ ]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

## Limpiar conjuntos de datos

Luego, limpie los conjuntos de datos. El código que se muestra a continuación borrará los conjuntos de datos de interacciones y elementos.

In [ ]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

## Limpiar esquemas

Luego, limpie los esquemas. Se crea un esquema de elementos y uno de interacciones y se eliminan.

In [ ]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

## Limpiar grupos de conjuntos de datos

Por último, limpie el grupo de conjunto de datos:

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

## Limpiar bucket de S3 y rol de IAM

En primer lugar, elimine el rol, luego vacíe el bucket y elimínelo.

In [ ]:
iam = boto3.client('iam')

Identifique el nombre del rol que desea eliminar.

No puede eliminar un rol de IAM que aún tiene políticas adjuntas. Después de que identifique el rol relevante, enumere las políticas adjuntas a ese rol.

In [ ]:
iam.list_attached_role_policies(
    RoleName = role_name
)

Debe desconectar las políticas en el resultado anterior mediante el código que aparece a continuación. Repita este paso para cada política adjunta.

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

Por último, debería poder eliminar el rol de IAM.

In [ ]:
iam.delete_role(
    RoleName = role_name
)

Para eliminar un bucket de S3, primero debe estar vacío. La forma más sencilla de eliminar un bucket de S3 es simplemente navegar a S3 en la consola de AWS, eliminar los objetos en el bucket y, luego, eliminar el bucket de S3.